In this notebook we will look how we can use Cython to generate a faster callback and hopefully shave off some running time from our integartion.

In [ ]:
import json
import time
import numpy as np
from odesys import ODEsys
from chem import odesys_from_reactions_names_and_params

The `ODEsys` class and convenience functions from previous notebook (35) has been put in two modules for easy importing. Recapping what we did last:

In [ ]:
watrad_data = json.load(open('radiolysis_300_Gy_s.json'))
watrad = odesys_from_reactions_names_and_params(**watrad_data)
tout = np.logspace(-6, 3, 200)  # close to one hour of operation
c0 = {'H2O': 55.4e3, 'H+': 1e-4, 'OH-': 1e-4}
y0 = [c0.get(symb.name, 0) for symb in watrad.y]

In [ ]:
tic = time.time()
yout, info = watrad.integrate_odeint(tout, y0)
toc = time.time()
print(toc - tic)

so that is the benchmark to beat, we will export our expressions as Cython code:

In [ ]:
cython_template = """
cimport numpy as cnp
import numpy as np

def f(cnp.ndarray[cnp.float64_t, ndim=1] y, double t, %(args)s):
    cdef cnp.ndarray[cnp.float64_t, ndim=1] out = np.empty(y.size)
    %(f_exprs)s
    return out
    
def j(cnp.ndarray[cnp.float64_t, ndim=1] y, double t, %(args)s):
    cdef cnp.ndarray[cnp.float64_t, ndim=2] out = np.empty((y.size, y.size))
    %(j_exprs)s
    return out

"""

We then subclass `ODEsys` to have it render, compile and import the code:

In [ ]:
import sympy as sp
import pyximport
pyximport.install()

class ODEaw(ODEsys):
    _counter = [0]
    def setup(self):
        self._counter[0] += 1
        mod_name = 'ode_cython_%d' % self._counter[0]
        idxs = list(range(len(self.f)))
        subs = {s: sp.Symbol('y[%d]' % i) for i, s in enumerate(self.y)}
        f_exprs = ['out[%d] = %s' % (i, str(self.f[i].xreplace(subs))) for i in idxs]
        j_exprs = ['out[%d, %d] = %s' % (ri, ci, self.j[ri, ci].xreplace(subs)) for ri in idxs for ci in idxs]
        ctx = dict(
            args=', '.join(map(str, self.p)),
            f_exprs = '\n    '.join(f_exprs),
            j_exprs = '\n    '.join(j_exprs),
        )
        open('%s.pyx' % mod_name, 'wt').write(cython_template % ctx)
        mod = __import__(mod_name)
        self.f_eval = mod.f
        self.j_eval = mod.j

In [ ]:
cython_sys = odesys_from_reactions_names_and_params(ODEcls=ODEaw, **watrad_data)

In [ ]:
cython_sys._counter

In [ ]:
tic = time.time()
yout, info = cython_sys.integrate_odeint(tout, y0)
toc = time.time()
print(toc - tic)

That is a considerable speed up from before. But the solver still has to
allocate memory for creating new arrays at each call, and each evaluation
has to pass the python layer which is now the bottleneck for the integration.

In order to speed up integration further we need to make sure the solver can evaluate the function and jacobian without calling into Python.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Just to see that everything looks alright:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
cython_sys.plot_result(tout, yout, info, ax=ax)
ax.set_xscale('log')
ax.set_yscale('log')